In [23]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [24]:
import numpy as np
# генерируем данные
np.random.seed(42)
sz = 100
x = np.random.rand(sz, 1)
y = 1 + 2 * x * x * x * x+ 0.1 * np.random.randn(sz, 1)

x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

In [30]:
from torch import nn
from torch.optim import SGD

#задаем модель и параметры обучения
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1, 1)).to(device)
print(model.state_dict())
print(model.parameters())

lr = 0.01
n_epochs = 10000
loss_fn = nn.MSELoss(reduction='mean')
optimizer = SGD(model.parameters(), lr=lr)

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])
<generator object Module.parameters at 0x79f1f74a4eb0>


In [26]:
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

# задаем датасеты и загрузчики
dataset = TensorDataset(x_tensor, y_tensor)
train_dataset, val_dataset = random_split(dataset, [80, 20])

train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

In [27]:
# задаем шаг обучения
def make_train_step(model, loss_fn, optimizer):

    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    return train_step

train_step = make_train_step(model, loss_fn, optimizer)

In [31]:
# обучаем модель
for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)

In [32]:
# тестируем модель
with torch.no_grad():
    for x_val, y_val in val_loader:
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        model.eval()
        yhat = model(x_val)
        val_loss = loss_fn(yhat, y_val)

print(model.state_dict())
print(loss, val_loss)

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])
0.08277549594640732 tensor(0.2854)
